# Báo cáo giữa kì Project
Lớp TTNT-156727 , Nhóm G18

# 1. Thông tin chung

### Thành viên
- Phạm Quốc Minh 20225743
- Nguyễn Thành Bách 20235660
- Trịnh Hữu An 20225593
- Lê Bá Ngọc Hiểu 20225627

### Lịch thực hiện
- W25: Đăng ký nhóm
- W26: Đề xuất project (1/3)
- W31: Báo cáo tiến độ giữa kỳ (5/4)
- W37: Hoàn thành và gửi báo cáo project (17/5)
- W38-40: Trình bày project, Q&A

# 2.Tiến độ giữa kì

## Bài toán
                      
                        **:Game Pacman:** 👻                           

![Pacman](https://media.vietnamplus.vn/images/7255a701687d11cb8c6bbc58a6c80785c531738e3787169ce34b631b27454b96eb5d7f68434f98ca1ca38a1124cee423/PacMan.jpg)


*Trong trò chơi này PACMAN sẽ **tự động** tìm thức ăn bằng các thuật toán được cài đặt sẵn với môi trường là một mê cung được khởi tạo gồm có tường , food và có thể có các Monster truy đuổi.*

## Phương pháp
⏭ *Game sẽ có 4 level khác nhau* và ta sẽ dùng các thuật toán khác nhau để giải quyết từng level




**🧭Level 1**:

*   Pacman thu thập thức ăn trong mê cung, không có Monster
---
 => Dùng **BFS** để tìm min_path tới food

**🔍Level 2**:

*   Tầm nhìn của Pacman bị giới hạn trong phạm vi 3 ô vuông.Pacman có thể nhìn thấy 8 ô xung quanh và mở rộng phạm vi nhìn thêm 3 ô nữa.
*   Monster di chuyển từng bước một xung quanh vị trí ban đầu. Mỗi bước của Pacman di chuyển, Monster cũng thay đổi vị trí.
---
=>Dùng thuật toán tìm kiếm cục bộ :**Heuristic local search** để tìm đường đi cho Pacman.




**🧠 Level 3**

Trong level này, **Pacman** sử dụng thuật toán **Monte Carlo Tree Search (MCTS)** để cân nhắc nhiều khả năng và mô phỏng tương lai.  
Trong khi đó, các con **ma** được điều khiển bởi thuật toán **A***, luôn tìm đường ngắn nhất và hiệu quả nhất để săn đuổi Pacman.  
Một bên là chiến lược dựa trên xác suất, một bên là sự săn đuổi quyết liệt dựa trên mục tiêu.


**⚖️Level 4**:

* Monsters sẽ truy đuổi nhằm tiêu diệt Pacman. Pacman sẽ cố gắng ăn nhiều thức ăn nhất có thể.
* Pacman sẽ thua cuộc nếu va chạm phải Monster
* Monsters có thể đi xuyên qua nhau.
* Với mỗi bước Pacman di chuyển, Monsters cũng di chuyển.
---
=>Dùng thuật toán **Minimax** để tìm đường đi cho Pacman 
=>Dùng thuật toán **A*** để di chuyển Monster.

## Phân công
- Phạm Quốc Minh: Cài đặt thuật toán BFS để tìm đường đi ngắn nhất đến food trong Level 1, thiết kế cấu trúc game.
- Nguyễn Thành Bách: Xây dựng thuật toán Heuristic Local Search và Model-Based Monte Carlo Tree Search cho Pacman , xây dựng hệ thống điều khiển Pacman.
- Trịnh Hữu An: Cài đặt thuật toán A* giúp Monster đuổi theo Pacman trong Level 3,thiết kế các object cho game , quản lý tiến độ dự án.
- Lê Bá Ngọc Hiểu: Cài đặt thuật toán Minimax cho Pacman trong Level 3, thiết kế UI/UX.

## Chương trình


### BFS

In [ ]:
from Utils.Utils import find_nearest_food, moving, isValid

def BFS(_map, food_positions, start_row, start_col, N, M):
    visited = [[False for _ in range(M)] for _ in range(N)]
    trace = [[[-1, -1] for _ in range(M)] for _ in range(N)]

    [food_row, food_col, index] = find_nearest_food(food_positions, start_row, start_col)

    if food_row == -1:
        return []
    
    lt = []
    chk = False
    visited[start_row][start_col] = True
    lt.append([start_row, start_col])
    while len(lt) > 0:
        [row, col] = lt.pop(0)

        if row == food_row and col == food_col:
            chk = True
            break

        for i in range(4):
            new_row = row + moving[i][0]
            new_col = col + moving[i][1]
            if isValid(_map, new_row, new_col, N, M) and not visited[new_row][new_col]:
                visited[new_row][new_col] = True
                trace[new_row][new_col] = [row, col]
                lt.append([new_row, new_col]) 

    if not chk:
        food_positions.pop(index)
        return BFS(_map, food_positions, start_row, start_col, N, M)
    
    result = [[food_row, food_col]]
    [row, col] = trace[food_row][food_col]
    while row != -1:
        result.insert(0, [row, col])
        [row, col] = trace[row][col]

    return result

### Local Search

In [ ]:
from Utils.Utils import moving,isWall
from constants import FOOD,MONSTER, WALL

def update_cost(_map, pos, current_pos, N, M, depth, visited, object_type, cost):
    """Cập nhật điểm số cho các ô dựa trên loại đối tượng và khoảng cách"""
    start_row, start_col = pos
    current_row, current_col = current_pos
    visited.append((current_row, current_col))

    if depth < 0 or pos == current_pos:
        return

    # Tính điểm dựa vào loại đối tượng và khoảng cách
    point = 0
    if object_type == FOOD:
        if depth == 2 :
            point = 35
        elif depth == 1 :
            point =10
        else:
            point =5

    if object_type == MONSTER:
        point = float("-inf") if depth > 0 else -100

    cost[current_row][current_col] += point

    # Lan truyền ảnh hưởng đến các ô lân cận
    for d_r, d_c in moving:
        new_row, new_col = current_row + d_r, current_col + d_c
        if isWall(_map, new_row, new_col, N, M) and (new_row, new_col) not in visited:
            update_cost(_map, pos, (new_row, new_col), N, M, depth - 1, visited.copy(), object_type, cost)


def scan_surroundings(_map, pos, current_pos, N, M, depth, visited, cost, visited_penalty):
    """Quét xung quanh để tìm thức ăn và quái vật, cập nhật điểm số"""
    start_row, start_col = pos
    current_row, current_col = current_pos
    visited.append((current_row, current_col))

    if depth <= 0:
        return

    # Khám phá các hướng có thể đi
    for d_r, d_c in moving:
        new_row, new_col = current_row + d_r, current_col + d_c
        if isWall(_map, new_row, new_col, N, M) and (new_row, new_col) not in visited:
            # Nếu gặp thức ăn hoặc quái vật, lan truyền ảnh hưởng
            object_pos = (new_row, new_col)
            if _map[new_row][new_col] == FOOD:
                update_cost(_map, pos, object_pos, N, M, 2, [], FOOD, cost)
            elif _map[new_row][new_col] == MONSTER:
                update_cost(_map, pos, object_pos, N, M, 2, [], MONSTER, cost)

            # Tiếp tục khám phá sâu hơn
            scan_surroundings(_map, pos, object_pos, N, M, depth - 1, visited.copy(), cost, visited_penalty)

    # Trừ điểm các ô đã đi qua trước đó
    cost[current_row][current_col] -= visited_penalty[current_row][current_col]


def local_search(_map, start_row, start_col, N, M, visited_penalty):
    """Tìm bước đi tiếp theo tốt nhất cho Pacman"""
    cost = [[0 for _ in range(M)] for _ in range(N)]
    visited = []

    # Quét và đánh giá các ô xung quanh
    scan_surroundings(_map, (start_row, start_col), (start_row, start_col), N, M, 3, visited, cost, visited_penalty)

    # Tìm ô kề có điểm cao nhất
    max_score = float("-inf")
    best_move = []

    for d_r, d_c in DDX:
        new_row, new_col = start_row + d_r, start_col + d_c
        if _map[new_row][new_col] != WALL and cost[new_row][new_col] > max_score:
            max_score = cost[new_row][new_col]
            best_move = [new_row, new_col]

    return best_move

### Model-Based Monte Carlo Tree Search 

In [ ]:
from Utils.utils import *
import math
import random


class Node:
    def __init__(self, prev, action):
        self.prev = prev
        self.prevaction = action
        self.childList = []
        self.visitnumber = 0
        self.score = 0
        self.expanded = False

    def createChild(self, action):
        childNode = Node(self, action)
        self.childList.append(childNode)
        return childNode


def gameEvaluation(rootState, currentState):
    # Đánh giá trạng thái game
    # Trả về điểm số - có thể cần điều chỉnh dựa vào cách đánh giá trạng thái của bạn
    if currentState.isWin():
        return 1000
    elif currentState.isLose():
        return -1000
    else:
        # Đánh giá dựa trên điểm số, khoảng cách đến thức ăn, v.v.
        return currentState.getScore() - rootState.getScore()


class GameState:
    def __init__(self, _map, food_Position, pacman_row, pacman_col, N, M, Score=0):
        self.map = _map.copy()
        self.food_Position = food_Position.copy()
        self.pacman_row = pacman_row
        self.pacman_col = pacman_col
        self.N = N
        self.M = M
        self.Score = Score

    def getLegalPacmanActions(self):
        # Trả về danh sách các hành động hợp lệ
        actions = []
        directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]  # left, right, up, down
        action_names = ["LEFT", "RIGHT", "UP", "DOWN"]

        for i in range(4):
            new_row = self.pacman_row + directions[i][0]
            new_col = self.pacman_col + directions[i][1]

            if 0 <= new_row < self.N and 0 <= new_col < self.M and self.map[new_row][
                new_col] != 1:  # Nếu không phải là tường
                actions.append(action_names[i])

        return actions

    def generatePacmanSuccessor(self, action):
        # Tạo trạng thái mới sau khi thực hiện hành động
        new_row, new_col = self.pacman_row, self.pacman_col

        if action == "LEFT":
            new_col -= 1
        elif action == "RIGHT":
            new_col += 1
        elif action == "UP":
            new_row -= 1
        elif action == "DOWN":
            new_row += 1

        # Kiểm tra nếu là vị trí hợp lệ
        if 0 <= new_row < self.N and 0 <= new_col < self.M and self.map[new_row][new_col] != 1:
            new_state = GameState(self.map, self.food_Position.copy(), new_row, new_col, self.N, self.M, self.Score)

            # Kiểm tra nếu ăn được thức ăn
            food_found = False
            for i, pos in enumerate(self.food_Position):
                if pos[0] == new_row and pos[1] == new_col:
                    new_state.food_Position.pop(i)
                    new_state.Score += 10  # Điểm cho việc ăn thức ăn
                    food_found = True
                    break

            return new_state

        return None

    def isWin(self):
        # Thắng nếu ăn hết thức ăn
        return len(self.food_Position) == 0

    def isLose(self):
        # Thua nếu bị ma bắt (cần triển khai logic phát hiện ma)
        # Đơn giản hóa: giả sử không có ma, không bao giờ thua
        return False

    def getScore(self):
        return self.Score


def backUp(node, score):
    while node.prev is not None:
        node.visitnumber += 1
        node.score += score
        node = node.prev
    node.visitnumber += 1  # Cập nhật cho nút gốc


def defaultPolicy(node, rootState):
    # Mô phỏng trò chơi từ trạng thái hiện tại
    nodeSet = []
    tempNode = node

    while tempNode.prev is not None:
        nodeSet.append(tempNode)
        tempNode = tempNode.prev

    nodeSet.reverse()
    tempState = rootState

    # Tái tạo trạng thái hiện tại
    for i in nodeSet:
        prevState = tempState
        tempState = tempState.generatePacmanSuccessor(i.prevaction)
        if tempState is None:
            backUp(i, gameEvaluation(rootState, prevState))
            return None
        if tempState.isWin():
            backUp(i, gameEvaluation(rootState, tempState))
            return None
        if tempState.isLose():
            backUp(i, gameEvaluation(rootState, tempState))
            return None

    # Mô phỏng một số bước ngẫu nhiên
    for j in range(5):  # Mô phỏng 5 bước
        if not (tempState.isWin() or tempState.isLose()):
            legal = tempState.getLegalPacmanActions()
            if not legal:
                break
            prevState = tempState
            tempState = tempState.generatePacmanSuccessor(random.choice(legal))
            if tempState is None:
                return None
        else:
            break

    return gameEvaluation(rootState, tempState)


def bestChild(node, exploration_weight=1.0):
    curr_max = -float('inf')
    best_children = []

    for child in node.childList:
        if child.visitnumber > 0:  # Tránh chia cho 0
            # UCB1 formula
            exploitation = child.score / child.visitnumber
            exploration = exploration_weight * math.sqrt((2 * math.log(node.visitnumber)) / child.visitnumber)
            value = exploitation + exploration

            if value > curr_max:
                curr_max = value
                best_children = [child]
            elif value == curr_max:
                best_children.append(child)

    if best_children:
        return random.choice(best_children)
    return None


def expansion(node, rootState):
    # Mở rộng node bằng cách thêm một node con mới
    nodeSet = []
    action_list = []
    tempNode = node

    while tempNode.prev is not None:
        nodeSet.append(tempNode)
        tempNode = tempNode.prev

    nodeSet.reverse()
    tempState = rootState

    # Tái tạo trạng thái hiện tại
    for i in nodeSet:
        prevState = tempState
        tempState = tempState.generatePacmanSuccessor(i.prevaction)
        if tempState is None:
            backUp(i, gameEvaluation(rootState, prevState))
            return None
        if tempState.isWin():
            backUp(i, gameEvaluation(rootState, tempState))
            return None
        if tempState.isLose():
            backUp(i, gameEvaluation(rootState, tempState))
            return None

    # Lấy danh sách hành động đã được thử
    for child in node.childList:
        action_list.append(child.prevaction)

    # Thêm một hành động mới
    legal_actions = tempState.getLegalPacmanActions()
    for action in legal_actions:
        if action not in action_list:
            child_node = node.createChild(action)

            if len(node.childList) == len(legal_actions):
                node.expanded = True

            return child_node

    # Nếu tất cả hành động đã được thử
    node.expanded = True
    return None


def treePolicy(root_node, root_state):
    node = root_node

    while True:
        if not node.expanded:
            return expansion(node, root_state)
        else:
            next_node = bestChild(node)
            if next_node is None:
                return node
            node = next_node


def MCTS(map, food_Position, start_row, start_col, N, M, iterations=100):
    # Khởi tạo trạng thái gốc
    root_state = GameState(map, food_Position, start_row, start_col, N, M)
    root_node = Node(None, None)
    root_node.visitnumber = 1  # Để tránh chia cho 0 trong UCB1

    # Chạy thuật toán MCTS
    for _ in range(iterations):
        expanded_node = treePolicy(root_node, root_state)
        if expanded_node is None:
            continue

        score = defaultPolicy(expanded_node, root_state)
        if score is not None:
            backUp(expanded_node, score)

    # Chọn hành động tốt nhất
    best_actions = []
    max_visits = -1

    for child in root_node.childList:
        if child.visitnumber > max_visits:
            max_visits = child.visitnumber
            best_actions = [child.prevaction]
        elif child.visitnumber == max_visits:
            best_actions.append(child.prevaction)

    if not best_actions:
        # Nếu không có hành động nào, chọn một hành động hợp lệ ngẫu nhiên
        legal_actions = root_state.getLegalPacmanActions()
        if legal_actions:
            return random.choice(legal_actions)
        return None

    # Trả về hành động tốt nhất
    return best_actions[random.randint(0, len(best_actions) - 1)]


## Vấn đề gặp phải

- **Chưa thể hoàn thiện kịp phần UI/UX** để demo cho giữa kỳ.
- **Chưa hoàn thiện thuật toán A*** để đảm bảo tính hiệu quả và tối ưu trong việc tìm đường cho các con ma.
- **Chưa tích hợp đầy đủ các thuật toán vào hệ thống**: Các thuật toán như MCTS và A* chưa được kết nối hoàn chỉnh với gameplay.
- **Chưa hoàn thiện phần tương tác người dùng**: Các điều khiển hoặc thông báo chưa được làm rõ ràng, gây khó khăn cho người chơi trong việc theo dõi tiến trình.
